In [18]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

base_url = "https://www.airlinequality.com/airline-reviews/turkish-airlines/page/2/?sortby=post_date%3ADesc&pagesize=100"
proceed = True
all_reviews_data = []

while proceed:
    url = base_url
    page = requests.get(url)
    soup = BeautifulSoup(page.text, "html.parser")

    # Check if the page is valid
    if soup.title and "404 Not Found" in soup.title.text:
        proceed = False
        break
    
    all_reviews = soup.find_all("div", class_="tc_mobile")
    for reviews in all_reviews:
        # Extract the message
        text_content = reviews.find_all("div", class_="text_content")
        message = [content.text.strip() for content in text_content]

        # Extract review stats
        review_stats = reviews.find("div", class_="review-stats")
        if review_stats:
            stats_data = {}
            rows = review_stats.find_all("tr")
            for row in rows:
                header = row.find("td", class_="review-rating-header")
                value = row.find("td", class_="review-value")
                if header and value:
                    stats_data[header.text.strip()] = value.text.strip()

            # Combine message and stats into a single dictionary
            review_dict = {
                'message': message,
                **stats_data  # Unpack the stats_data dictionary
            }
            all_reviews_data.append(review_dict)

    proceed = False  # Set to False to stop after one page for testing

# Convert to a Pandas DataFrame for easy analysis
df = pd.DataFrame(all_reviews_data)

# Display the first 30 rows
print(df.head(30))

# Save the data to a CSV file
df.to_csv("British_airlines_reviews_data.csv", index=False)

                                              message Type Of Traveller  \
0   [✅ Trip Verified | I am writing to express my ...    Family Leisure   
1   [✅ Trip Verified |  My review/feedback to the ...    Couple Leisure   
2   [✅ Trip Verified | I have always rated Turkish...    Family Leisure   
3   [✅ Trip Verified |   Turkish Airlines is the w...          Business   
4   [✅ Trip Verified |   Our flight was delayed fo...    Family Leisure   
5   [✅ Trip Verified |   I had an awful experience...    Family Leisure   
6   [✅ Trip Verified | I recently traveled with Tu...      Solo Leisure   
7   [✅ Trip Verified | Turkish Airlines has a grea...      Solo Leisure   
8   [✅ Trip Verified |   To begin with, the Turkis...      Solo Leisure   
9   [Not Verified | My little boy didn't recieve t...    Family Leisure   
10  [Not Verified | This was one of the best flyin...      Solo Leisure   
11  [✅ Trip Verified |   Once a great airline but ...    Couple Leisure   
12  [Not Verified |   I r